In [1]:
import pandas as pd
from glob import glob
import os



In [2]:
files = glob('data/model_evals/chicken/*csv')
files = [os.path.basename(i) for i in files]

In [3]:
files

['DeepSEA_model_0.0001_eval.csv',
 'DeepSEA_model_0.001_eval.csv',
 'DanQ_model_0.0005_eval.csv',
 'DeepSEA_model_0.00005_eval.csv',
 'DeepSEA_model_0.0005_eval.csv',
 'DanQ_model_0.001_eval.csv',
 'DanQ_model_0.0001_eval.csv',
 'DanQ_model_0.00005_eval.csv']

In [63]:
def read_track_names(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df.columns = ['track_name']
    df['type'] = [i.split('_')[0].replace('ChIP-', '') for i in df['track_name']]
    df.loc[df['type'].isin(['DNaseSeq', 'ATAC']), 'type'] = 'ATAC/DNaseSeq'
    df.loc[df['type'].isin(['CTCF', 'DMC1']), 'type'] = 'CTCF/DMC1'
    return list(df['type']), list(df['track_name'])

In [18]:
#read_track_names('data/data_info/cattle_colnames.txt')

In [76]:
#a,b = read_track_names('data/data_info/salmon_colnames.txt')

In [115]:
def read_res(paths, spec):
    res = []
    for fi in paths:
        tokens = fi.split('_')
        method = tokens[0]
        lr = tokens[2]
        lr = f"{float(lr):.0e}".replace('e-0', 'e-')
        fi = f'data/model_evals/{spec}/{fi}'
        track_fi = f'data/data_info/{spec}_colnames.txt'
        types, tracks = read_track_names(track_fi)
        df = pd.read_csv(fi)
        df['Spec'] = spec.capitalize()
        df['Method'] = method
        df['LR'] = lr
        df['Type'] = types
        df['Track_name'] = tracks
        ## exclude ATAC_Gill - they are low quality
        #pick = ['ATAC_Gill' in i for i in tracks]
        #df = df[~pd.Series(pick)]
        res.append(df)
    return pd.concat(res)


In [116]:
a = read_res(files, 'chicken')

In [117]:
specs = ['cattle', 'pig', 'chicken', 'salmon']

In [118]:
res = [read_res(files, s) for s in specs]

In [119]:
all_specs = pd.concat(res)

In [120]:
all_specs.to_csv('out_tables/model_performance.csv', index=False)

In [121]:
all_specs.shape

(4584, 7)